In [1]:
import pandas as pd
df18= pd.read_pickle('df18featuesy.p')


In [2]:
features = df18.columns

In [5]:
features = features.drop('price')

In [8]:
import tensorflow as tf

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

D:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [15]:
from sklearn.model_selection import train_test_split, KFold

dataX = df18[features].values
dataY = df18[['price']].values

X_train, X_test, y_train, y_test = train_test_split(dataX, dataY)

# y_train = y_train.reshape(-1,1)
# y_test = y_test.reshape(-1,1)

ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)


tf.reset_default_graph()

X = tf.placeholder(dtype=tf.float32, 
                   shape=(None,X_train.shape[1]),
                   name='X')
y = tf.placeholder(dtype=tf.float32,
                   shape=(None),
                   name='y')
n_hidden = 100
hidden1 = tf.layers.dense(X, units=n_hidden,
                          name='hidden1',
                          activation=tf.nn.relu)
y_hat = tf.layers.dense(hidden1, units=1,
                        name='y_hat',
                        activation=tf.nn.sigmoid)

loss = tf.losses.log_loss(y, y_hat)

optimizer = tf.train.AdamOptimizer(0.001)

training_op = optimizer.minimize(loss)


init = tf.global_variables_initializer()

test_err = []
train_err = []

with tf.Session() as sess:
    init.run()
    
    for epoch in range(2000):
        sess.run(training_op, 
                 feed_dict={X:X_train, y:y_train})
        train_loss = sess.run(loss, feed_dict={X:X_train, y:y_train})
        test_loss = sess.run(loss, feed_dict={X:X_test, y:y_test})
        test_err.append(test_loss)
        train_err.append(train_loss)
        if epoch % 100 == 0:
            print('epoch', epoch+1, train_loss, test_loss)
        
    pred = sess.run(y_hat, feed_dict={X:X_test})
    
metrics.r2_score(y_test, pred)

epoch 1 0.8470708 0.8378967
epoch 101 -0.08079441 -0.083593935
epoch 201 -0.21692261 -0.22133781
epoch 301 -0.23605873 -0.23890565
epoch 401 -0.24457967 -0.24491407
epoch 501 -0.25064972 -0.24861266
epoch 601 -0.25512257 -0.25192797
epoch 701 -0.25856107 -0.25476465
epoch 801 -0.2614203 -0.25707772
epoch 901 -0.26388013 -0.2590063
epoch 1001 -0.2660334 -0.26065117
epoch 1101 -0.26793057 -0.26197815
epoch 1201 -0.2697305 -0.26311833
epoch 1301 -0.27146465 -0.26403746
epoch 1401 -0.27312994 -0.26475894
epoch 1501 -0.27472052 -0.2652981
epoch 1601 -0.27623275 -0.26578468
epoch 1701 -0.2777004 -0.26611358
epoch 1801 -0.27913645 -0.26633906
epoch 1901 -0.280562 -0.2665419


-0.2050419276641171